In [8]:
import pandas as pd
from geopy.geocoders import MapBox
from tqdm.auto import tqdm

tqdm.pandas()

geolocator = MapBox(api_key="pk.eyJ1IjoibWljaGFlbGh0aG9tYXMiLCJhIjoiY205M2RkaTN6MGg3YzJqb2x3NW1mN2MwaSJ9.6uRZ-TIfxHjfuPTTbwOZig")

leases = pd.read_csv("../data/Leases.csv")

full_addresses = (
    leases["address"]
    + ", "
    + leases["city"]
    + ", "
    + leases["state"]
    + " "
    + leases["zip"].apply(lambda x: str(int(x)) if pd.notna(x) else "")
)

address_info = pd.DataFrame.from_dict(
    {"id": leases["building_id"], "address": full_addresses}
).drop_duplicates()

In [9]:
def geocode(address):
  location = geolocator.geocode(address, exactly_one=True)
  if location is None:
    return pd.Series([None, None])

  return pd.Series([location.latitude, location.longitude])

# address_info[['latitude', 'longitude']] = address_info['address'].progress_apply(lambda x: geocode(x))

# address_info.to_csv('address_info.csv')

In [11]:
address_info

,id,address,latitude,longitude
0,Atlanta_Central Perimeter_Atlanta_10 Glenlake ...,"10 Glenlake Pky NE, Atlanta, GA 30328",33.936269,-84.361239
1,Atlanta_Northwest_Atlanta_100 City View_3330 C...,"3330 Cumberland Blvd, Atlanta, GA 30339",33.873792,-84.458516
2,Atlanta_Northwest_Atlanta_1000 Parkwood_1000 P...,"1000 Parkwood Cir SE, Atlanta, GA 30339",33.901861,-84.459599
3,Atlanta_Northwest_Atlanta_1100 Circle 75_1100 ...,"1100 Circle 75 Pky SE, Atlanta, GA 30339",33.889732,-84.465435
4,Atlanta_Central Perimeter_Atlanta_1200 Ashwood...,"1200 Ashwood Pky, Atlanta, GA 30338",33.933749,-84.340570


In [17]:
market_locations = pd.DataFrame.from_dict({"market": leases['market'].unique() + ", United States"})

market_locations[['latitude', 'longitude']] = market_locations['market'].progress_apply(lambda x: geocode(x))

100%|██████████| 29/29 [00:06<00:00,  4.67it/s]


In [19]:
market_locations.to_csv('../data/market_locations.csv')

In [21]:
leases.columns

Index(['year', 'quarter', 'monthsigned', 'market', 'building_name',
       'building_id', 'address', 'region', 'city', 'state', 'zip',
       'internal_submarket', 'internal_class', 'leasedSF', 'company_name',
       'internal_industry', 'transaction_type', 'internal_market_cluster',
       'costarID', 'space_type', 'CBD_suburban', 'RBA', 'available_space',
       'availability_proportion', 'internal_class_rent', 'overall_rent',
       'direct_available_space', 'direct_availability_proportion',
       'direct_internal_class_rent', 'direct_overall_rent',
       'sublet_available_space', 'sublet_availability_proportion',
       'sublet_internal_class_rent', 'sublet_overall_rent', 'leasing'],
      dtype='object')

In [ ]:
pre_leased = leases[leases['year'] == 2018].groupby(['market'])['leasedSF'].sum()
post_leased = leases[leases['year'] == 2024].groupby(['market'])['leasedSF'].sum()

change = (post_leased - pre_leased) / pre_leased
change = change.reset_index()
change.columns = ['market', 'growth_rate']

growth_rates = pd.merge(change, market_locations, on='market')

growth_rates.to_csv('../data/growth_rates.csv')

,market,growth_rate,latitude,longitude
0,Atlanta,0.069421,33.750800,-84.389854
1,Austin,-0.274204,30.268072,-97.742805
2,Baltimore,0.065072,39.289642,-76.612300
3,Boston,-0.278465,42.358880,-71.056800
4,Charlotte,-0.180741,35.227207,-80.843090
5,Chicago,-0.285539,41.881954,-87.632360
6,Chicago Suburbs,0.170250,41.881954,-87.632360
7,Dallas/Ft Worth,-0.124231,32.762771,-96.855290
8,Denver,-0.189764,39.741680,-104.986110
9,Detroit,0.164173,42.331398,-83.046220
